In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# --- 1. Data Generation/Loading (Simulated Data) ---
# In a real-world scenario, you would load your historical sales and stock data here.
# For demonstration, we'll create synthetic data for a single SKU.

def generate_synthetic_data(num_days=365 * 2, sku_id='SKU001'):
    """Generates synthetic historical demand and stockout data."""
    np.random.seed(42) # for reproducibility

    dates = pd.date_range(start='2023-01-01', periods=num_days, freq='D')
    df = pd.DataFrame({'Date': dates})

    # Base demand with some seasonality
    df['DayOfYear'] = df['Date'].dt.dayofyear
    df['Demand'] = (10 + 5 * np.sin(df['DayOfYear'] * (2 * np.pi / 365)) +
                    np.random.normal(0, 2, num_days)).round().astype(int)
    df['Demand'] = df['Demand'].apply(lambda x: max(0, x)) # Ensure demand is non-negative

    # Simulate stock levels and stockouts
    initial_stock = 100
    stock_on_hand = [initial_stock] * num_days
    stockout_event = [0] * num_days # 1 if stockout occurred, 0 otherwise

    # Simple reorder logic for simulation: reorder when stock falls below 30
    reorder_threshold = 30
    reorder_quantity = 50

    for i in range(1, num_days):
        current_demand = df.loc[i, 'Demand']
        prev_stock = stock_on_hand[i-1]

        if prev_stock < current_demand:
            stockout_event[i] = 1 # Stockout occurred
            stock_on_hand[i] = 0 # Stock depleted
        else:
            stock_on_hand[i] = prev_stock - current_demand

        # Simulate reorder after 3 days lead time if stock was low
        if i >= 3 and stock_on_hand[i-3] <= reorder_threshold and stockout_event[i-3] == 0:
            stock_on_hand[i] += reorder_quantity # Stock arrives

    df['StockOnHand'] = stock_on_hand
    df['StockoutEvent'] = stockout_event
    df['SKU_ID'] = sku_id

    # Calculate actual sales (min(demand, stock_available))
    df['Sales'] = df.apply(lambda row: min(row['Demand'], row['StockOnHand'] + row['Demand'] - row['Demand']), axis=1)
    df['Sales'] = df.apply(lambda row: min(row['Demand'], row['StockOnHand'] + (row['Demand'] if row['StockoutEvent'] == 0 else 0)), axis=1)
    # A more accurate way to derive sales from demand and stock:
    # If stockout, sales = stock_available_before_demand, else sales = demand
    df['Sales'] = np.where(df['StockoutEvent'] == 1, df['StockOnHand'].shift(1).fillna(initial_stock), df['Demand'])
    df['Sales'] = df['Sales'].apply(lambda x: max(0, x)) # Ensure sales are non-negative

    # Let's refine the stockout and sales logic for clarity
    df['StockBeforeDemand'] = df['StockOnHand'].shift(1).fillna(initial_stock)
    df['ActualSales'] = np.minimum(df['Demand'], df['StockBeforeDemand'])
    df['StockoutEvent'] = (df['Demand'] > df['StockBeforeDemand']).astype(int)

    # The target variable for forecasting: next day's demand
    df['TargetDemand'] = df['Demand'].shift(-1) # Shift demand up to predict next day's demand

    return df

print("Generating synthetic data...")
df = generate_synthetic_data(num_days=365 * 3) # 3 years of data
print("Synthetic data generated successfully.")
print(df.head())
print(df.info())

# --- 2. Feature Engineering ---
print("\nPerforming feature engineering...")
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
df['DayOfWeek'] = df['Date'].dt.dayofweek # Monday=0, Sunday=6
df['DayOfMonth'] = df['Date'].dt.day
df['DayOfYear'] = df['Date'].dt.dayofyear

# Lagged features for demand and stockout trends
# These features help the model understand recent patterns
for lag in [1, 7, 14, 30]:
    df[f'Demand_Lag_{lag}'] = df['Demand'].shift(lag)
    df[f'Stockout_Lag_{lag}'] = df['StockoutEvent'].shift(lag)
    df[f'Sales_Lag_{lag}'] = df['ActualSales'].shift(lag)

# Moving averages of demand
for window in [7, 30]:
    df[f'Demand_MA_{window}'] = df['Demand'].rolling(window=window, min_periods=1).mean().shift(1)

# Simple "event spike" simulation: e.g., higher demand around holidays (simplified)
# Let's assume a spike around Christmas (day 355-365) and mid-year (day 170-180)
df['IsHolidaySpike'] = ((df['DayOfYear'] >= 355) | (df['DayOfYear'] <= 10) |
                        ((df['DayOfYear'] >= 170) & (df['DayOfYear'] <= 180))).astype(int)

# Drop rows with NaN values created by shifting (for initial lags)
df.dropna(inplace=True)
print("Feature engineering complete.")
print(df.head())

# --- 3. Model Training (XGBoost) ---
print("\nTraining XGBoost model...")

# Define features (X) and target (y)
features = [col for col in df.columns if col not in ['Date', 'SKU_ID', 'Demand', 'StockOnHand', 'StockoutEvent', 'TargetDemand', 'ActualSales', 'StockBeforeDemand']]
X = df[features]
y = df['TargetDemand']

# Split data into training and testing sets
# Use a time-based split to simulate real-world forecasting
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
df_test = df[train_size:].copy() # Keep original df_test for plotting

print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")

# Initialize and train XGBoost Regressor
# Parameters can be tuned for better performance
model = xgb.XGBRegressor(objective='reg:squarederror',
                         n_estimators=100,
                         learning_rate=0.1,
                         max_depth=5,
                         subsample=0.8,
                         colsample_bytree=0.8,
                         random_state=42,
                         n_jobs=-1)

model.fit(X_train, y_train)
print("XGBoost model trained successfully.")

# --- 4. Forecasting ---
print("\nMaking predictions...")
predictions = model.predict(X_test)
predictions = np.maximum(0, predictions).round().astype(int) # Ensure non-negative and integer predictions
df_test['PredictedDemand'] = predictions
print("Predictions made.")

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"RMSE on test set: {rmse:.2f}")
print(f"This RMSE of {rmse:.2f} indicates the average magnitude of the errors in predicting demand.")
print(f"For a typical demand range, an RMSE of {rmse:.2f} might be acceptable, but further tuning could improve it.")


# --- 5. Apply Moving Average Smoothing and Plot Optimal Restocking Thresholds ---
print("\nCalculating optimal restocking thresholds and plotting...")

# Calculate a rolling average of predicted demand for smoothing
# This helps in setting more stable restocking thresholds
df_test['SmoothedPredictedDemand'] = df_test['PredictedDemand'].rolling(window=7, min_periods=1).mean()

# Define optimal restocking threshold based on smoothed predicted demand
# This is a simplified approach. In reality, this would involve lead times,
# service level targets, holding costs, stockout costs, etc.
# For simplicity, let's say we want to have enough stock for 1.5 times the smoothed daily demand
# plus a safety stock buffer.
safety_stock_factor = 0.5 # Additional buffer
lead_time_days = 3 # Example lead time

# Restocking Threshold = (Smoothed Predicted Daily Demand * Lead Time) + Safety Stock
df_test['OptimalRestockThreshold'] = (df_test['SmoothedPredictedDemand'] * lead_time_days) + \
                                      (df_test['SmoothedPredictedDemand'] * safety_stock_factor)
df_test['OptimalRestockThreshold'] = df_test['OptimalRestockThreshold'].round().astype(int)

# --- 6. Visualization ---
plt.figure(figsize=(16, 8))
sns.lineplot(x='Date', y='Demand', data=df_test, label='Actual Daily Demand', color='blue', alpha=0.7)
sns.lineplot(x='Date', y='PredictedDemand', data=df_test, label='Predicted Daily Demand', color='orange', linestyle='--', alpha=0.7)
sns.lineplot(x='Date', y='OptimalRestockThreshold', data=df_test, label='Optimal Restocking Threshold', color='green', linestyle='-', linewidth=2)
sns.lineplot(x='Date', y='StockoutEvent', data=df_test, label='Stockout Event (1=Yes)', color='red', marker='x', alpha=0.5)

plt.title(f'Inventory Optimization for {df_test["SKU_ID"].iloc[0]}: Demand Forecast and Restocking Thresholds')
plt.xlabel('Date')
plt.ylabel('Units')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()

print("\n--- Summary and Recommendations ---")
print(f"The model predicts future demand based on historical patterns, seasonality, and recent trends.")
print(f"The 'Optimal Restocking Threshold' line indicates the recommended stock level at which a reorder should be triggered to avoid stockouts, considering predicted future demand and a buffer.")
print(f"By keeping stock above this threshold, the system aims to minimize stockouts while optimizing inventory holding costs.")
print(f"\nNext Steps:")
print(f"- Integrate with real historical sales and inventory data.")
print(f"- Refine feature engineering (e.g., external factors like promotions, competitor actions).")
print(f"- Tune XGBoost hyperparameters for improved accuracy.")
print(f"- Implement a more sophisticated inventory policy model considering lead times, service levels, and costs.")
print(f"- Extend to multiple SKUs and handle varying demand patterns.")